In [ ]:
library(GSVA)
library(GSEABase)
library(limma)
library(msigdbr)
library(org.Mm.eg.db)
library(Seurat)
library(ComplexHeatmap)
library(tidyverse)

In [ ]:
gsva_results = readRDS("gsva_results_group.rds")
gsva_adult = readRDS("gsva_adult_group.rds")
gsva_ep = readRDS("gsva_ep_group.rds")

In [ ]:
pa = cluster::pam(gsva_results, k = 10)
cluster = rep(x = c("Fascia", "Inflammatory", "Oxidative_stress", "Myo", "Fascia", "Inflammatory", "Oxidative_stress"), each = 30)

In [ ]:

clust_hm <- Heatmap(gsva_results,
                    left_annotation = rowAnnotation(foo = anno_block(gp = gpar(fill = 2:11),
                                      labels_gp = gpar(col = "white", fontsize = 10))),
                    top_annotation = columnAnnotation(cluster = cluster,
                                      annotation_legend_param = list(title="cluster",labels_gp=gpar(col = "black"))),
                    row_split = pa$clustering, 
                    cluster_rows = FALSE,
                    cluster_columns = FALSE,
                    #right_annotation = ha,
                    show_row_names = FALSE,
                    show_column_names = FALSE,
                    column_order = c(paste0("EP_", 90:1), paste0("Adult_", 1:120)) ,
                    # column_split = (c((90:1)+120,1:120)-1) %/% 30,
                    # column_gap = unit(0.1, "mm")
                    name = "clust_hm",
                    show_row_dend = FALSE,
                    #row_order = ordered_index
                    )
p=draw(clust_hm)


In [ ]:
rowlist = row_order(p)
custom_order <- c("4", "5", "6", "7", "9", "8", "2", "3", "10", "1")

In [ ]:
pa$clustering = factor(pa$clustering,levels=custom_order)

In [ ]:
library(viridis)
library(RColorBrewer)
library(circlize)

In [ ]:
library(ggplot2)

In [ ]:
white_theme <- function() {
  theme(panel.background = element_rect(fill = "black"),
        plot.background = element_rect(fill = "black",color="black"),
        legend.background = element_rect(fill = "black"),
        panel.grid = element_blank(),
        axis.line = element_line( linewidth = 0.5,color="white"),
        axis.text = element_text(color="white"),
        axis.title = element_text(color="white"),
        axis.ticks = element_line(color = "white"),
       )
}

In [ ]:

# 创建一个空的数据框来存储所有数据
all_data <- data.frame()

# 循环处理每个分组
for (n in seq(1, 10)) {
  gsva_adult1 <- gsva_adult[pa$clustering == n, ]
  adult_mat_mean <- colMeans(gsva_adult1)
  gsva_ep1 <- gsva_ep[pa$clustering == n, ]
  ep_mat_mean <- c(colMeans(gsva_ep1), rep(NA, times = 30))
  
  # 创建一个数据框并添加分组信息
  data <- data.frame(
    x = seq(1, 120),
    ep = ep_mat_mean,
    ad = adult_mat_mean,
    group = n
  )
  
  # 将数据框合并到all_data中
  all_data <- rbind(all_data, data)
}

# 使用ggplot2绘制分面图
p <- ggplot(all_data, aes(x = x)) +
  geom_point(aes(y = ep),size=0.8, color = '#42B236') +
  geom_smooth(aes(y = ep), color = '#42B236') +
  geom_point(aes(y = ad),size=0.8, color = '#8A67F9') +
  geom_smooth(aes(y = ad), color = '#8A67F9') +
  facet_wrap(~ group, ncol = 1, scales="free_y") +  # 这里可以调整ncol来控制每行的图数
  theme_classic() + white_theme()+
  ggtitle("Cluster Plots")

# 展示图形
options(repr.plot.width = 4, repr.plot.height =25)
print(p)


In [ ]:
ggsave(plot=p,"GSVA_lineplot_c10.pdf",w=4,h=25)